In [1]:
import re
import string
from pickle import dump
from unicodedata import normalize
from numpy import array
from keras.preprocessing.text import Tokenizer
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

Using TensorFlow backend.


In [38]:
# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

In [2]:
# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

In [3]:
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# # load dataset
# filename = 'parallel/deu.txt'
# doc = load_doc(filename)
# # split into english-german pairs
# pairs = to_pairs(doc)
# # clean sentences
# cp = clean_pairs(pairs)
# # save clean pairs to file
# save_clean_data(cp, 'old.pkl')
# # spot check

filename = 'tat.en-hi.en'
eng = load_doc(filename).strip().split('\n')

filename = 'tat.en-hi.hi'
hin = load_doc(filename).strip().split('\n')


pairs = []

# for i in range(len(eng)):
#     pairs.append([eng[i], hin[i]])
# split into english-german pairs




In [4]:
# line = pairs[500][1]
# line = line.split()
re_print = re.compile('[^%s]' % re.escape(string.printable))
table = str.maketrans('', '', string.punctuation)
eng2 = []
for line in eng:
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    # tokenize on white space
    line = line.split()
    # convert to lowercase
    line = [word.lower() for word in line]
    # remove punctuation from each token
    line = [word.translate(table) for word in line]
    # remove non-printable chars form each token
    line = [re_print.sub('', w) for w in line]
    # remove tokens with numbers in them
    line = [word for word in line if word.isalpha()]
    eng2.append(' '.join(line))


In [5]:
pairs = list()
for i in range(len(eng2)):
    pair = list()
    pair.append(eng2[i])
    pair.append(hin[i])
    pairs.append(pair)


In [6]:
def get_only_small(pairs, MAX_LEN=10, MAX_LEN2=20):
    new_pairs = list()
    for pair in pairs:
        line = pair[0] #english line maxlength
        if(len(line.strip().split(' ')) <= MAX_LEN and len(pair[1].strip().split(' ')) <= MAX_LEN2):
            pair2 = list()
            pair2.append(pair[0])
            pair2.append(pair[1])
            new_pairs.append(pair2)
    return new_pairs

In [7]:
cp = get_only_small(pairs, 10, 10)

In [8]:
print(len(cp))
cp = array(cp)
save_clean_data(cp, 'hindi_tat.pkl')

4033
Saved: hindi_tat.pkl


In [9]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('hindi_tat.pkl')
shuffle(raw_dataset)
# reduce dataset size
n_sentences = len(raw_dataset)
n_train = int(n_sentences*0.9)
dataset = raw_dataset[:n_sentences]
# random shuffle

# split into train/test
train, test = dataset[:n_train], dataset[n_train:]
# save
save_clean_data(dataset, 'hindi_tat-both.pkl')
save_clean_data(train, 'hindi_tat-train.pkl')
save_clean_data(test, 'hindi_tat-test.pkl')

Saved: hindi_tat-both.pkl
Saved: hindi_tat-train.pkl
Saved: hindi_tat-test.pkl


In [10]:
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# load datasets
dataset = load_clean_sentences('hindi_tat-both.pkl')
train = load_clean_sentences('hindi_tat-train.pkl')
test = load_clean_sentences('hindi_tat-test.pkl')

# dataset = load_clean_sentences('english-german-10-10-both.pkl')
# train = load_clean_sentences('english-german-10-10-train.pkl')
# test = load_clean_sentences('english-german-10-10-test.pkl')

In [11]:

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer


In [12]:
dataset[0]

array(['does your sister live there', 'क्या तुम्हारी बहन वहां रहती है?'],
      dtype='<U65')

In [13]:
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [14]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

English Vocabulary Size: 2840
English Max Length: 10
German Vocabulary Size: 3857
German Max Length: 10


In [15]:
dataset[:, 1]

array(['क्या तुम्हारी बहन वहां रहती है?', 'बहुत महंगी है!',
       'क्या टॉम मुझे देखना चाहता है?', ..., 'हम नदी के पार तेर पाए.',
       'तुमने क्या जवाब दिया?', 'वह कहानी सुन कर पागल हो गया।'],
      dtype='<U65')

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [31]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)


# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y


# load datasets
dataset = load_clean_sentences('hindi-both.pkl')
train = load_clean_sentences('hindi-train.pkl')
test = load_clean_sentences('hindi-test.pkl')

# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

eng_length = ger_length
print('Set eng_length to ger_length')
# prepare training data
print('Encoding trainX')
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
print('Encoding trainY...')
trainY = encode_output(trainY, eng_vocab_size)
print('Done')
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
print('Encoding testY...')
testY = encode_output(testY, eng_vocab_size)
print('Done')


English Vocabulary Size: 15567
English Max Length: 10
German Vocabulary Size: 21617
German Max Length: 10
Set eng_length to ger_length
Encoding trainX
Encoding trainY...
Done
Encoding testY...
Done


In [32]:
len(testY)

7736

In [24]:
# define NMT model
from keras.layers import RNN
from keras.layers import Bidirectional
from custom_recurrents import AttentionDecoder

def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units, return_sequences=False))
#     model.add(AttentionDecoder(256, 256))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model


# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 128)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model_30march_hindi_tat.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 50)            192850    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 10, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 2840)          144840    
Total params: 378,090
Trainable params: 378,090
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model.fit(trainX, trainY, epochs=10, batch_size=2, validation_data=(testX[:1000], testY[:1000]), callbacks=[checkpoint], verbose=1)

Train on 3629 samples, validate on 404 samples
Epoch 1/10
3629/3629 [==============================] - 83s 23ms/step - loss: 3.4644 - val_loss: 3.9687
Epoch 2/10
3629/3629 [==============================] - 81s 22ms/step - loss: 3.3696 - val_loss: 4.0142
Epoch 3/10
 516/3629 [===>..........................] - ETA: 1:07 - loss: 3.2097

KeyboardInterrupt: 

In [20]:
len(trainX[0])

17

In [22]:
from keras.models import load_model
source = testX[0]
from numpy import argmax

# load model
# model = load_model('model_27march_mymodel2.h5')
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	source = source.reshape((1, source.shape[0]))
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None


In [23]:
for i in range(len(test)-20, len(test)):
    print(test[i][0],'|', test[i][1], '|', predict_sequence(model, eng_tokenizer, testX[i]))

i dived into the river | मैंने नदी में छलाँग लगाकर डुबकी ली। | i is is to
i dont want to lose you tom | मैं तुम्हें खोना नहीं चाहती, टॉम। | i is is to
listening to him she got tired | उनकी बात सुनते-सुनते वे थक गयीं। | i is is to
try to jump as high as possible | जितना ऊँचा कूद सकते हो कूदो। | i is is
i was able to win the first prize | मैंने प्रथम पुरस्कार जीत लिया। | i is is
our class consists of fifty boys | हमारी क्लास में पचास लड़के हैं। | i is is to
is everything ready | सब कुछ तैयार है? | the
its the law | यही कानून है. | i is
what is the population of india | भारत की जनसंख्या क्या है? | i is is to
he expressed his feelings for nature in a poem | उसने प्रकृति के प्रति अपनी भावनाओं को प्रकट किया। | i is is
your journey starts here | तुम्हारा सफ़र यहीं शुरू होता है। | the
your daughter is not a child any more | तुम्हारी बेटी अब बच्ची नहीं है। | i is
he lost everything | उसने सब कुछ खोया। | i is
he is smelling the soup | वह सूप को सूँघ रहा है। | i is is
he came to see you yesterday

In [49]:
text = "Er sagte, es sei beabsichtigt, eine Kontamination zu vermeiden"
mine = encode_sequences(ger_tokenizer, ger_length, [text])
predict_sequence(model, eng_tokenizer, mine[0])

'he said it was to a haircut'

In [58]:
import numpy as np
np.save('ger_tokenizer', ger_tokenizer)
np.save('eng_tokenizer', eng_tokenizer)

array([ 22,   3, 105, 588,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [50]:
testX[0]

array([   8,  196,   16,  305,   25,    1,  855,   34,  100, 5006,   15,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [14]:
# evaluate the skill of the model
from nltk.translate.bleu_score import corpus_bleu
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
#       source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
#       if i < 10:
#           print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append(raw_target.split())
        predicted.append(translation.split())
        if i>1000:
            break
#   actual = [[['this', 'is', 'a', 'test']], [['this', 'is', 'a', 'test']]]
#   predicted = [['this', 'is', 'a', 'test'], ['this', 'is', 'a', 'test']]
    actual2 = []
    for a in actual:
        actual2.append([a])
    actual = actual2

    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [15]:
evaluate_model(model, eng_tokenizer, testX, test)

BLEU-1: 0.363639
BLEU-2: 0.209455
BLEU-3: 0.137074
BLEU-4: 0.056273


In [98]:
# 1-gram individual BLEU
from nltk.translate.bleu_score import sentence_bleu
reference = [['meet', 'me', 'in', 'an', 'hour']]
candidate = ['me', 'in' ,'an' ,'hour']
score = sentence_bleu(reference, candidate,weights=(0.25, 0.25,0.25, 0.25))
print(score)

0.7788007830714049
